In [2]:
import random

In [3]:
SEARCH_SPACE = {
    'k_size_a': [1, 3, 5, 7],
    'k_size_b': [1, 3, 5, 7],
    'out_channels_a': [8, 16, 32, 64],
    'out_channels_b': [8, 16, 32, 64],
    'include_pool_a': [True, False],
    'include_pool_b': [True, False],
    'pool_type_a': ['max_pooling','avg_pooling'],
    'pool_type_b': ['max_pooling','avg_pooling'],
    'activation_type_a': ['relu', 'tanh', 'elu', 'selu'],
    'activation_type_b': ['relu', 'tanh', 'elu', 'selu'],
    'include_b': [True, False],
    'include_BN_a': [True, False],
    'include_BN_b': [True, False],
    'skip_connection': [True, False],
}

In [4]:
class Generation():
  def __init__(self,
               fit_survival_rate: float,
               unfit_survival_rate: float,
               mutation_rate: float,
               pop_size: int,
               phase: int,
               search_space: dict,
               prev_best: Chromosome):
    self.fit_survival_rate = fit_survival_rate
    self.unfit_survival_rate = unfit_survival_rate
    self.mutation_rate = mutation_rate
    self.pop_size = pop_size
    self.pop = []

    for i in range(pop_size):
      self.pop.append(Chromosome(phase = phase,
                            prev_best = prev_best,
                            genes = self.make_gene(search_space)))
      
  def make_gene(search_space: dict):
    gene = {}
    keys = search_space.keys()
    for key in keys:
      gene[key] = random.choice(search_space[key])

    return gene

  def sort_pop(self):
    sorted_pop = sorted(self.pop,
                        key = lambda x: x.fitness)
    self.pop = sorted_pop

  def generate(self):
    num_fit_selected = int(self.fit_survival_rate * self.pop_size)
    num_unfit_selected = int(self.unfit_survival_rate * self.pop_size)
    num_mutate = int(self.mutation_rate * self.pop_size)

    new_pop = []

    for i in range(num_fit_selected):
      new_pop.append(self.pop[i])

    for i in range(num_unfit_selected):
      new_pop.append(self.pop[self.pop_size - i - 1])

    if(num_mutate > len(new_pop)):
      indices_to_mutate = random.sample(range(0, len(new_pop)), num_mutate)
    else:
      indices_to_mutate = random.sample(range(0, len(new_pop)), len(new_pop))

    for i in indices_to_mutate:
      new_pop[i].mutate()

    parents_list = []
    for i in range(self.pop_size):
      parents = random.sample(range(0, len(new_pop)), 2)
      parents_list.append(tuple(parents))

    for p1, p2 in parents_list:
      new_pop.append(new_pop[p1].crossover(new_pop[p2]))
    
    self.pop = new_pop
  
  def find_fittest(self):
    self.sort_pop()
    return self.pop[0]


NameError: name 'Chromosome' is not defined